# Παίζοντας Atari με βαθιά ενισχυτική μάθηση

Προσοχή: για να μπορέσει να τρέξει αυτό το notebook στο Kaggle πρέπει να ενεργοποιήσετε το Internet και την GPU για τον πυρήνα σας. Αυτό απαιτεί την επιβεβαίωση του κινητού σας με SMS. Πηγαίντε στο δεξί sidebar στα settings και θα το δείτε. 

Τελικά πρέπει να έχετε αυτή την εικόνα:

![](https://i.imgur.com/Ek5hOIo.jpg)

## Tensorboard και ngrok

Αντικαταστήστε το authentication token σας που παίρνετε από το [ngrok](https://ngrok.com/) στη γραμμή που υποδεικνύεται. Όταν εκτελεστεί το κελί θα σας δώσει το URL όπου μπορείτε να βλέπετε το TensorBoard. Σημειώστε ότι σε περίπτωση επανεκκίνησης του πυρήνα θα πρέπει να ξανατρέξετε το κελί. Η διεύθυνση θα είναι διαφορετική, αλλά τα προηγούμενα στατιστικά σας δεν χάνονται (μέχρι να ανακυκλωθεί ο πυρήνας).

In [1]:
!apt install apt-utils psmisc -y
!killall -9 ngrok
!rm -rf *ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir -p ngrok-dir
!mv ngrok ngrok-dir/
# Στην επόμενη γραμμή βάλτε το δικό σας authentication token από το ngrok
!1pWBSAQUjPIhGSp7ilcTGKqhvC8_4PwMkw4KwFHtNFd6UeFx5

LOG_DIR = 'tb_log/'
!mkdir -p LOG_DIR

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('ngrok-dir/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print("Wait 10-15 seconds and then click the URL above to open TensorBoard")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apt libapt-pkg5.0
Suggested packages:
  apt-doc aptitude | synaptic | wajig
The following NEW packages will be installed:
  apt-utils psmisc
The following packages will be upgraded:
  apt libapt-pkg5.0
2 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 2267 kB of archives.
After this operation, 1053 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libapt-pkg5.0 amd64 1.6.12ubuntu0.2 [806 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 apt amd64 1.6.12ubuntu0.2 [1201 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 apt-utils amd64 1.6.12ubuntu0.2 [207 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 psmisc amd64 23.1-1ubuntu0.1 [52.5 kB]
Fetched 2267 kB in 0s (6381 kB/s) 
debconf: delaying package

## Εγκατάσταση βιβλιοθήκης και gym

In [2]:
!pip install --upgrade pip
!pip install --upgrade stable_baselines3[extra]
# we need a specific version of gym because of this issue: https://github.com/DLR-RM/stable-baselines3/issues/294
!pip install gym==0.17.3

     |████████████████████████████████| 1.5 MB 859 kB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.0
    Uninstalling pip-21.0:
      Successfully uninstalled pip-21.0
     |████████████████████████████████| 152 kB 865 kB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 864 kB/s eta 0:00:01


  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654653 sha256=b047737141783e754b3347adcd2b90ca172471baf3a23992b6113b9478133382
  Stored in directory: /root/.cache/pip/wheels/d1/81/4b/dd9c029691022cb957398d1f015e66b75e37637dda61abdf58
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.18.0
    Uninstalling gym-0.18.0:
      Successfully uninstalled gym-0.18.0


## Ορισμός παιχνιδιού

Βάλτε εδώ το string που αντιστοιχεί στο παιχνίδι σας. Για την ονοματολογία των περιβαλλόντων:
- v0 vs v4: v0 has repeat_action_probability of 0.25 (meaning 25% of the time the previous action will be used instead of the new action), while v4 has 0 (always follow your issued action)
- Deterministic: a fixed frameskip of 4, while for the env without Deterministic, frameskip is sampled from [2,4]
- There is also NoFrameskip-v4 with no frame skip and no action repeat stochasticity.

Αναλυτικότερα στην εκφώνηση της άσκησης.

In [3]:
atari_env_name='MsPacmanDeterministic-v4'

## Δημιουργία περιβάλλοντος

In [4]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

In [5]:
# Με τις συναρτήσεις που ακολουθούν κάνουμε την ίδια προεπεξεργασία με την Deepmind

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [6]:
import datetime # For filenames while logging

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

## Φόρτωση εκπαιδευμένου μοντέλου

Το Kaggle δεν μας αφήνει να ανεβάσουμε άλλου είδους δεδομένα εκτός από datasets. Θα χρησιμοποιήσουμε έναν οποιονδήποτε λογαριασμό Dropbox για να δημιουργήσουμε ένα απευθείας link για το αρχείο zip του εκπαιδευμένου μοντέλου.
1. Τοποθετήστε το zip μέσα σε ένα φάκελο Dropbox
2. Αν είστε σε Windows κάντε δεξί κλικ πάνω στο αρχείο και επιλέξτε "Copy Dropbox Link'. Αν είστε σε Linux μπείτε στην web εφαρμογή και κάντε το ίδιο (λέγεται "Copy link"
3. Κάντε κάπου paste το URL https://www.dropbox.com/s/umw5k9wcuiy0l2u/a2c_pong.zip?dl=0 και αλλάξτε το τελευταίο 0 σε 1 όπως εδώ https://www.dropbox.com/s/umw5k9wcuiy0l2u/a2c_pong.zip?dl=1
4. Αντιγράψτε το νέο URL στο clipboard.

Εδώ φέρνουμε ένα μοντέλο Α2C που έχουμε εκπαιδεύσει νωρίτερα. Εσείς απλά αλλάζετε το URL της παρακάτω εντολής για τα δικά σας αρχεία. Προσέξτε μετά την παράμετρο -Ο να βάλετε το σωστό όνομα αρχείου, με ποιο  όνομα θα σωθεί δηλαδή.

In [8]:
!wget --no-check-certificate 'https://www.dropbox.com/s/6dyw6weag29kai1/dqn_best_model.zip?dl=0' -O dqn_pacman.zip

--2021-03-13 11:37:40--  https://www.dropbox.com/s/6dyw6weag29kai1/dqn_best_model.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6dyw6weag29kai1/dqn_best_model.zip [following]
--2021-03-13 11:37:40--  https://www.dropbox.com/s/raw/6dyw6weag29kai1/dqn_best_model.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc709cb3867697eb03176321cd0f.dl.dropboxusercontent.com/cd/0/inline/BKnWIPSWK7lmDH9LYOwTkXwnBNh_chNBB8qcAYDYxHdQ8Mvj0g4nGeamL0QCV5COukqbvbMmyv3fepARzBupZ1jhOvGsDshYI8vzz4o_BVgA6hXy3mdPij7wVd0WUX4qU5qck8UIjSuyyRJea_QncbaR/file# [following]
--2021-03-13 11:37:41--  https://uc709cb3867697eb03176321cd0f.dl.dropboxusercontent.com/cd/0/inline/BKnWIPSWK7lmDH9LYOwTkXwnBNh_chNBB8qcAYDYxHdQ8Mvj0g4nGeamL0QCV5COukq

In [9]:
from stable_baselines3 import DQN
dqn_model = DQN.load("dqn_pacman", verbose=1, train_freq=2)

In [10]:
# Προσοχή, το n_envs πρέπει να είναι πάντα το ίδιο (αυτό με το οποίο έγινε η αρχική εκπαίδευση του μοντέλου)
new_env = make_atari_env(atari_env_name, n_envs=1)
# Frame-stacking with 4 frames
new_env = VecFrameStack(new_env, n_stack=4)
# Το μοντέλο δεν έχει μαζί του το περιβάλλον, πρέπει να του το αναθέσουμε ξανά.
dqn_model.set_env(new_env)

Wrapping the env in a VecTransposeImage.


In [19]:
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 4396.0 (+/-963.0700909071987)


## Καταγραφή video του actual gameplay του πράκτορα

Ωραία τα διαγράμματα, ωραίαο το TensorBoard, αλλά θέλουμε οπωσδήποτε να δούμε τον πράκτορα να παίζει το παιχνίδι!

In [12]:
!apt-get install ffmpeg freeglut3-dev xvfb  -y # For visualization

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
The following additional packages will be installed:
  freeglut3 libdrm-dev libegl-mesa0 libegl1 libgbm1 libgl1-mesa-dev libgles1
  libgles2 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev
  libice-dev libopengl0 libpthread-stubs0-dev libsm-dev libwayland-server0
  libx11-dev libx11-doc libx11-xcb-dev libxau-dev libxcb-dri2-0-dev
  libxcb-dri3-dev libxcb-glx0-dev libxcb-present-dev libxcb-randr0
  libxcb-randr0-dev libxcb-render0-dev libxcb-shape0-dev libxcb-sync-dev
  libxcb-xfixes0-dev libxcb1-dev libxdamage-dev libxdmcp-dev libxext-dev
  libxfixes-dev libxshmfence-dev libxt-dev libxxf86vm-dev mesa-common-dev
  x11proto-core-dev x11proto-damage-dev x11proto-dev x11proto-fixes-dev
  x11proto-xext-dev x11proto-xf86vidmode-dev xorg-sgml-doctools xtrans-dev
Suggeste

Preparing to unpack .../03-x11proto-core-dev_2018.4-4_all.deb ...
Unpacking x11proto-core-dev (2018.4-4) ...
Selecting previously unselected package libxau-dev:amd64.
Preparing to unpack .../04-libxau-dev_1%3a1.0.8-1ubuntu1_amd64.deb ...
Unpacking libxau-dev:amd64 (1:1.0.8-1ubuntu1) ...
Selecting previously unselected package libxdmcp-dev:amd64.
Preparing to unpack .../05-libxdmcp-dev_1%3a1.1.2-3_amd64.deb ...
Unpacking libxdmcp-dev:amd64 (1:1.1.2-3) ...
Selecting previously unselected package xtrans-dev.
Preparing to unpack .../06-xtrans-dev_1.3.5-1_all.deb ...
Unpacking xtrans-dev (1.3.5-1) ...
Selecting previously unselected package libpthread-stubs0-dev:amd64.
Preparing to unpack .../07-libpthread-stubs0-dev_0.3-4_amd64.deb ...
Unpacking libpthread-stubs0-dev:amd64 (0.3-4) ...
Selecting previously unselected package libxcb1-dev:amd64.
Preparing to unpack .../08-libxcb1-dev_1.13-2~ubuntu18.04_amd64.deb ...
Unpacking libxcb1-dev:amd64 (1.13-2~ubuntu18.04) ...
Selecting previously uns

Selecting previously unselected package libx11-doc.
Preparing to unpack .../48-libx11-doc_2%3a1.6.4-3ubuntu0.3_all.deb ...
Unpacking libx11-doc (2:1.6.4-3ubuntu0.3) ...
Setting up libopengl0:amd64 (1.0.0-2ubuntu2.3) ...
Setting up freeglut3:amd64 (2.8.1-3) ...
Setting up libgles1:amd64 (1.0.0-2ubuntu2.3) ...
Setting up libxcb-randr0:amd64 (1.13-2~ubuntu18.04) ...
Setting up libxshmfence-dev:amd64 (1.3-1) ...
Setting up libpthread-stubs0-dev:amd64 (0.3-4) ...
Setting up libdrm-dev:amd64 (2.4.101-2~18.04.1) ...
Setting up xorg-sgml-doctools (1:1.11-1) ...
Setting up x11proto-dev (2018.4-4) ...
Setting up x11proto-damage-dev (1:2018.4-4) ...
Setting up xtrans-dev (1.3.5-1) ...
Setting up x11proto-xext-dev (2018.4-4) ...
Setting up libxdmcp-dev:amd64 (1:1.1.2-3) ...
Setting up libice-dev:amd64 (2:1.0.9-2) ...
Setting up libx11-doc (2:1.6.4-3ubuntu0.3) ...
Setting up libgles2:amd64 (1.0.0-2ubuntu2.3) ...
Setting up libglu1-mesa:amd64 (9.0.0-2.1build1) ...
Setting up libglvnd-core-dev:amd64 

In [13]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [14]:
video_folder = '/kaggle/working/videos'

In [15]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [16]:
import numpy as np
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(eval_env, model, video_length=500, prefix='', video_folder=video_folder):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  # Start the video at step=0 and record video_length steps
  env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = env.reset()
  for _ in range(video_length):
    #action, _ = model.predict(obs)
    #θα επιλέξουμε την ενέργεια με τις μεγαλύτερες πιθανότητες
    actions, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = env.step(actions)

  # Close the video recorder
  env.close()

In [20]:
record_video(test_env, dqn_model, video_length=5000, prefix='dqn_pacman')

Saving video to  /kaggle/working/videos/dqn_pacman-step-0-to-step-5000.mp4


In [21]:
show_videos(video_path = video_folder, prefix='dqn')